In [1]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib_metadata 8.7.0
    Uninstalling importlib_metadata-8.7.0:
      Successfully uninstalled importlib_metadata-8.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [2]:
import mlflow
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments,EarlyStoppingCallback
import torch
from datasets import load_dataset
from datetime import datetime
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_DATASETS_CACHE"] = "./hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "./transformers_cache"

mlflow.set_experiment("MLflow DistilBERT Sentiment Analysis")

dataset = load_dataset("yelp_review_full")
train_dataset = dataset["train"].shuffle(seed=42).select(range(100000))
val_dataset = dataset["test"].shuffle(seed=42).select(range(25000))

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")

base_model_id = "distilbert-base-uncased"
MAX_LENGTH = 512

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token or tokenizer.unk_token or tokenizer.cls_token

def tokenize_and_pad(sample):
    encodings = tokenizer(
        text=sample["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )
    encodings["labels"] = sample["label"]
    return encodings

train_dataset = train_dataset.map(tokenize_and_pad, batched=True, remove_columns=["text"])
val_dataset = val_dataset.map(tokenize_and_pad, batched=True, remove_columns=["text"])

train_dataset.set_format("torch")
val_dataset.set_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_id,
    num_labels=5
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.tensor(logits).argmax(dim=-1).numpy()
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

training_args = TrainingArguments(
    report_to="mlflow",
    run_name=f"distilbert-base-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    output_dir="outputs_distilbert",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    fp16=torch.cuda.is_available(),
    learning_rate=2e-5,
    lr_scheduler_type="constant",
    num_train_epochs=5,
    save_total_limit=1,
    save_strategy="epoch",
    logging_steps=100,
    eval_strategy="epoch",
    warmup_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

model.config.use_cache = False

trainer.train()


2025-07-07 19:04:34.073761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751915074.475045      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751915074.587466      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025/07/07 19:04:51 INFO mlflow.tracking.fluent: Experiment with name 'MLflow DistilBERT Sentiment Analysis' does not exist. Creating a new experiment.


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_19/1318725500.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.827700,0.798889,0.648440,0.646064,0.646453,0.648440
2,0.743500,0.775994,0.660440,0.660330,0.660891,0.660440
3,0.673400,0.789970,0.660880,0.659833,0.659809,0.660880


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=4689, training_loss=0.7634292654605734, metrics={'train_runtime': 8993.3637, 'train_samples_per_second': 55.597, 'train_steps_per_second': 0.868, 'total_flos': 3.9742345728e+16, 'train_loss': 0.7634292654605734, 'epoch': 3.0})